Import Necessary Modules and Libraries


In [ ]:
pip install scapy

In [ ]:
import os
import pandas as pd
import numpy as np
from scapy.all import rdpcap, wrpcap, IP, TCP, UDP,Ether
from scapy.all import *
from sklearn.model_selection import train_test_split  # for data splitting
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf  # for neural network training

STEP1: Reading and Analyzing the PCAP File

In [ ]:
# Analyze the packets in the PCAP file by iterating over each packet and printing a summary of each packet using the summary() method.

# Load the pcap file
# Load the attack details (assuming a similar structure)
# attack_details_df = pd.read_csv('Darkgate2.csv')

#packets = rdpcap('Darkgate2.pcap')  #provided by the professor

# Analyze packets
#for packet in packets[:]:
 #   print(packet.summary())

STEP 2: Generating Synthetic Attack Files

In [ ]:
import csv
import random
import pandas as pd
from scapy.all import rdpcap, wrpcap, IP, TCP, UDP, Ether

output_pcap_file = 'generatedattack.pcap'

def load_ip_addresses():
    # Predefined list of IP addresses I found online (which are generally used for malware)

    # pull the ip addresses from the darkgate filesprof sent me
    ip_addresses = [
        "103.251.167.20", "109.70.100.2", "109.70.100.65", "109.70.100.68",
        "109.70.100.70", "142.44.234.69", "147.0.191.243", "162.155.77.166",
        "162.229.195.242", "185.220.100.240", "185.220.100.241", "185.220.100.243",
        "185.220.100.248", "185.220.100.252", "185.220.101.2", "185.220.101.38",
        "185.220.101.81", "185.243.218.202", "185.246.188.73", "185.56.83.83",
        "192.42.116.175", "192.42.116.183", "192.42.116.186", "192.42.116.189",
        "192.42.116.191", "192.42.116.197", "193.35.18.105", "194.26.192.77",
        "208.105.35.90", "209.177.72.205", "23.137.251.61", "23.251.87.120",
        "24.250.126.2", "35.146.95.0", "45.138.16.42"  # Only IPv4 adresses, no IPv6
    ]
    return random.sample(ip_addresses, min(10, len(ip_addresses)))  # selects 10 IP addresses

def modify_packet(packet, ip_addresses, common_ports):
    # Interchangeably use a pair of IP addresses to resemble a real traffic
    src_ip, dst_ip = random.sample(ip_addresses, 2)
    if IP in packet:
        packet[IP].src = src_ip
        packet[IP].dst = dst_ip
    # Commonly observed TCP flags and their combinations
    tcp_flags_options = ['A', 'S', 'F', 'FA', 'PA', 'RA', 'SA']  # include more

    # Randomly select ports and TCP flags for TCP packets
    if TCP in packet:
        sport, dport = random.sample(common_ports, 2)
        packet[TCP].sport = sport
        packet[TCP].dport = dport
        packet[TCP].flags = random.choice(tcp_flags_options)
        packet[TCP].seq = random.randint(0, 10000) # this part the generator program has to learn it from the pcaps prof provided
        packet[TCP].ack = random.randint(0, 10000)
        packet[TCP].window = random.choice([8192, 65535, 1024]) # check if they are more available packet options

        # Adjusting packet length
        payload_sizes = [0, 64, 128, 256, 512, 1024]  # - include more if there are any avaialble - Example lengths in bytes
        payload_size = random.choice(payload_sizes)
        if payload_size > 0:
            payload = 'A' * payload_size  # Generates a string of 'A's of length `payload_size`
            packet = packet / Raw(load=payload)
    elif UDP in packet:
        sport, dport = random.sample(common_ports, 2)
        packet[UDP].sport = sport
        packet[UDP].dport = dport

    return packet

def generate_synthetic_attack(output_pcap_file):
    # Load IP addresses
    ip_addresses = load_ip_addresses()

    # Common web ports
    common_ports = [80, 443, 8080, 1024, 65535] #-pull from the pcap files provded by the Prof # include more ports

    # Generate 100 synthetic packets
    synthetic_packets = []
    for _ in range(100):
        packet = Ether()/IP()/TCP()  # Basic TCP/IP packet
        packet = modify_packet(packet, ip_addresses, common_ports)
        synthetic_packets.append(packet)

    # Write modified packets to a new PCAP file
    wrpcap(output_pcap_file, synthetic_packets)
    print(f"Synthetic attack PCAP file generated: {output_pcap_file}")

# Specify the output PCAP file path
output_pcap_file = 'generatedattack.pcap'

# Generate the synthetic attack PCAP file
generate_synthetic_attack(output_pcap_file)

# NOTE AFTER THE FIRST RUN, YOU NEED TO DELETE THE GENERATED FILE OR IT WILL RAISE AN ERROR (i can also fix this by adding a control look or if else, but no need for now)


Synthetic attack PCAP file generated: generatedattack.pcap
